## Card Game Suppress Seven

### Rules
    - Use the 52 cards in 4 colors: Heart, Diamond, Spade and Club.
    - Four players, each one gets 13 cards in the beginning.
    - The player who has Spade 7 on hand goes first.
    - The players must use the cards which have the same color and consecutive number to the cards on the board.
        - For example, if there is ♠7 on board, the next player must use ♠6, ♠8 or ♥7,♦7,♣7.
    - If a player has not card can be used, he/she needs to suppress one card. This card should not be seen by the other players until the end of this game.
    - The total points of the suppressed cards would be calculated at the end of the game, the one has the least points win.

In [1]:
import numpy as np
import names
import random
from collections import OrderedDict, deque, defaultdict

In [2]:
class CustomError(Exception):
    """Base class for exceptions in this module."""
    pass

In [3]:
class CardNumberError(CustomError):
    def __init__(self, message, number):
        self.message = message
        self.number = number
    def __str__(self):
        return f"The numer {self.number} is {self.message} than the allowed card number range: 1~52."
class CardNumberDisaster(CustomError):
    def __init__(self,card):
        self.card = card
    def __str__(self):
        return f"The card numer {self.card.name()} is illegal."

In [4]:
# It is the card class, there are 52 cards in total, 13 cards for each suit. 
# So that we use the id between 1 and 52 to initialize the card.
class Card:
    suit_map = {0: 'Heart',
                1: 'Diamond',
                2: 'Spade',
                3: 'Club'}
    def __init__(self, number, card_code=None):
        if number > 52:
            raise CardNumberError('larger', number)
        elif number < 1:
            raise CardNumberError('smaller', number)
        else:
            self.number = (number-1)%13+1     # it is the number of this card, no matter which suit, 1-13
            self.suit = Card.suit_map[(number-1)//13] # it is the suit, one of the four
            self.card_id = number              # it is the id of the card, each card has an unique id, 1-52
            self.name = self.name()
            self.left = None
            self.right = None
            self.previous = None
            self.available = False
            self.suppressed = False
            self.player_id = None
            self.card_code = number if card_code is None else card_code
            self.onboard = False
    def __str__(self):
        return f"This card is {self.suit} {self.number}. The uid of this card is {self.card_id}."
    
    def name(self):
        return f"{self.suit} {self.number}"

In [5]:
class Player:
    identity=0    # it will be accumlated for each player
    def __init__(self, name=None, email=None):
        Player.identity += 1
        self.name = name or names.get_first_name()
        self.email = email
        self.player_id = Player.identity
        self.hand_card = None
        self.decoder = {} # only the player himself/herself knows the code of his/her own cards
        self.available = []
        self.suppressed = {} # easier for the player to show suppressed card by suits
        self.score = 0

    def get_hand_cards(self, hand_card):
        for card in hand_card:
            card.player_id = self.player_id
            self.decoder[card.card_code] = card.card_id # All the card are represented by card id in the backend
        self.hand_card = OrderedDict(sorted([(card.card_id, card) for card in hand_card]))
    
    def _check_available(self):
        self.available = []
        for card_id, card in self.hand_card.items():
            if card.available:
                self.available.append(card_id)
                
    def show_available(self):
        send_text = "You current available cards are:\n"
        for card_id in self.available:
            send_text += (f"{self.hand_card[card_id].name} is available, " 
                          + f"use card code {self.hand_card[card_id].card_code} if you want to use.\n")
        return send_text
    
    # tell the player which cards are in hand, which are available or have to suppress
    def show_hand_cards(self):
        send_text = "You currently have:\n"
        suppress_text = "\nYou have to suppress: \n"
        hand_cards = defaultdict(list)
        suppressed_cards = defaultdict(list)
        for card_id in self.hand_card.keys():
            hand_cards[self.hand_card[card_id].suit].append(str(self.hand_card[card_id].number))
            suppress_text += (f"{self.hand_card[card_id].name} is in your hand, " 
                              + f"use card code {self.hand_card[card_id].card_code} if you want to suppress. \n")
        for card_id in self.suppressed.keys():
            suppressed_cards[self.suppressed[card_id].suit].append(str(self.suppressed[card_id].number))
        for suit in hand_cards.keys():
            send_text += f"{suit} {', '.join(hand_cards[suit])}.\n"
        send_text += '\nYou have suppressed: \n'
        for suit in suppressed_cards.keys():
            send_text += f"{suit} {', '.join(suppressed_cards[suit])}.\n"    
        if len(self.available) == 0:  # only if there is no card available, player is allowed to suppress card
            send_text += suppress_text
        return send_text 

In [6]:
class Game:
    identity=0
    def __init__(self, random_seed=123):
        Game.identity += 1
        self.game_id = Game.identity
        self.card_map = OrderedDict()    # all the calculation in the backend uses the id (card or player), 
        self.player_map = OrderedDict()  # need to use this map to link the id to the object
        self.card_pool = []
        self.players = deque([])   # Here the deque is being used, because the player with Spade 7 need to start, 
                                   # NOT based on the order of geting card
        pool = list(range(1,53))
        random.seed(random_seed)
        random.shuffle(pool)
        self.onboard_cards = []    # record the cards on board
        self.min_score = 9999      # after the last round, the player has the lowest score wins
        self.winner = None
        self.encoder = {}          # create the visiable code for each card, it is different from the card id, 
                                   #since we do not want the other players know which card is suppressed. 
                                   # It is different in different game.
        for i, v in enumerate(pool):
            self.encoder[i+1] = v
    def get_cards(self):
        previous_card = None
        """
        left means the card which number is -1 of current one, right means +1 of current card.
        However, for card.number < 7, +1 card is the previous card of current one, means if the previous card is not on the board, 
        the current one cannot be available. Since the +1 card already stored as the previous card, no need to set the right card.
        The same for card > 7, no need to set .left.
        """
        for i in range(1,53):
            this_card = Card(i, self.encoder[i])
            if this_card.number < 7:             
                this_card.left = previous_card    # left means the card which number is one less than the current one
                if previous_card is not None:
                    previous_card.previous = this_card  # previous card means, if previous one is not on the board, 
                                                        # this card cannot be used
            elif this_card.number > 7:
                this_card.previous = previous_card     #7 is the previous for both 6 and 8, here sets 8's.
                if previous_card is not None:
                    previous_card.right = this_card 
            elif this_card.number == 7:           # For all suits, 7 is always be available, and it is the previous for both 6 and 8, 
                                                  # here sets 6's. 
                this_card.left = previous_card
                previous_card.previous = this_card
                this_card.available = True      # set 7 to available

            else:
                raise CardNumberDisaster(this_card)  # not going to happen
            if this_card.number == 13:
                previous_card = None
            else:
                previous_card = this_card
            self.card_map[this_card.card_id] = this_card
        self.card_pool = list(self.card_map.values())   # list of card class objs
        random.shuffle(self.card_pool)
    
    def prepare(self):
        for i in range(4):     # make sure only four players
            this_name = input("Please input your name: ")
            this_name = None if this_name == '' else this_name
            this_email = input("Please input your email: ")
            this_email = None if this_email == '' else this_email
            this_player = Player(this_name, this_email)
            print(f"Welcome {this_player.name}, have fun!")
            this_player.get_hand_cards(self.card_pool[i*13:(i+1)*13])
            self.player_map[this_player.player_id] = this_player
            self.players.append(this_player)  # Here the players are added to the deque one by one
            
        init_player = self.player_map[self.card_map[33].player_id]  # The one has Spade 7 need to be the first player
        first_player_index = self.players.index(init_player)
        # Here the new order is rotated, the same order as assigning cards, but the first player has the Spade 7
        self.players.rotate(-1*first_player_index)
        
    def check_onboard(self, show=True):
        self.onboard_cards = []
        for card_id in self.card_map.keys():
            if self.card_map[card_id].onboard:
                self.onboard_cards.append(self.card_map[card_id])
        if show:
            current_onboard = defaultdict(list)
            for card in self.onboard_cards:
                current_onboard[card.suit].append(str(card.number))
            print("\nCurrent onboard cards are:")
            print("************************************************")
            for suit in current_onboard.keys():
                print(f"{suit} {', '.join(current_onboard[suit])}.")
            print("************************************************")

In [18]:
first_game = Game(324)

In [19]:
first_game.get_cards()

In [20]:
first_game.prepare()

Please input your name: 
Please input your email: 
Welcome Samantha, have fun!
Please input your name: 
Please input your email: 
Welcome Jose, have fun!
Please input your name: 
Please input your email: 
Welcome Wesley, have fun!
Please input your name: 
Please input your email: 
Welcome Leroy, have fun!


In [21]:
first_game.players[0].name

'Samantha'

In [22]:
first_game.players[0].hand_card

OrderedDict([(1, <__main__.Card at 0x1ec04770460>),
             (4, <__main__.Card at 0x1ec06403ee0>),
             (13, <__main__.Card at 0x1ec047865e0>),
             (14, <__main__.Card at 0x1ec047869d0>),
             (27, <__main__.Card at 0x1ec04786f10>),
             (31, <__main__.Card at 0x1ec04786e20>),
             (33, <__main__.Card at 0x1ec04786340>),
             (36, <__main__.Card at 0x1ec047868b0>),
             (37, <__main__.Card at 0x1ec047868e0>),
             (39, <__main__.Card at 0x1ec04786880>),
             (44, <__main__.Card at 0x1ec04786820>),
             (45, <__main__.Card at 0x1ec04786970>),
             (50, <__main__.Card at 0x1ec04786d30>)])

In [23]:
for card in first_game.players[0].hand_card.values():
    print(card)
    if card.available:
        print('it is available')

This card is Heart 1. The uid of this card is 1.
This card is Heart 4. The uid of this card is 4.
This card is Heart 13. The uid of this card is 13.
This card is Diamond 1. The uid of this card is 14.
This card is Spade 1. The uid of this card is 27.
This card is Spade 5. The uid of this card is 31.
This card is Spade 7. The uid of this card is 33.
it is available
This card is Spade 10. The uid of this card is 36.
This card is Spade 11. The uid of this card is 37.
This card is Spade 13. The uid of this card is 39.
This card is Club 5. The uid of this card is 44.
This card is Club 6. The uid of this card is 45.
This card is Club 11. The uid of this card is 50.


In [25]:
first_game.players[0].available

[]

In [26]:
first_game.players[0].show_available()

'You current available cards are:\n'

In [27]:
first_game.players[0]._check_available()

In [29]:
print(first_game.players[0].show_available())

You current available cards are:
Spade 7 is available, use card code 19 if you want to use.



In [30]:
first_game.players[1].show_available()

'You current available cards are:\n'

In [31]:
first_game.players[1]._check_available()

In [32]:
first_game.players[1].show_available()

'You current available cards are:\nHeart 7 is available, use card code 21 if you want to use.\n'

In [33]:
first_game.players[1].hand_card

OrderedDict([(2, <__main__.Card at 0x1ec047704f0>),
             (5, <__main__.Card at 0x1ec063f8f10>),
             (6, <__main__.Card at 0x1ec04786070>),
             (7, <__main__.Card at 0x1ec04786a30>),
             (10, <__main__.Card at 0x1ec04786fd0>),
             (15, <__main__.Card at 0x1ec047863d0>),
             (23, <__main__.Card at 0x1ec04786dc0>),
             (24, <__main__.Card at 0x1ec04786df0>),
             (28, <__main__.Card at 0x1ec04786f40>),
             (38, <__main__.Card at 0x1ec04786b80>),
             (40, <__main__.Card at 0x1ec04786640>),
             (47, <__main__.Card at 0x1ec04786040>),
             (49, <__main__.Card at 0x1ec04786a00>)])

In [35]:
first_game.players[2]._check_available()
print(first_game.players[2].show_available())

You current available cards are:
Diamond 7 is available, use card code 47 if you want to use.
Club 7 is available, use card code 35 if you want to use.



In [36]:
first_game.players[3]._check_available()
print(first_game.players[3].show_available())

You current available cards are:



In [55]:
first_game.players[0].decoder

{16: 34,
 1: 32,
 11: 39,
 36: 43,
 42: 41,
 37: 35,
 35: 44,
 46: 28,
 39: 33,
 22: 42,
 6: 50,
 13: 26,
 38: 1}

In [30]:
print(first_game.card_map[33])

This card is Spade 7. The uid of this card is 33.


In [17]:
first_game.card_map[33].previous

In [20]:
first_game.card_map[32].name

'Spade 6'

In [19]:
first_game.card_map[32].previous.name

'Spade 7'

In [21]:
first_game.card_map[34].name

'Spade 8'

In [22]:
first_game.card_map[34].previous.name

'Spade 7'

In [18]:
# test
test_card = Card(60)

CardNumberError: The numer 60 is larger than the allowed card number range: 1~52.

In [8]:
test_card

In [10]:
test_card.name()

'Diamond 7'

In [12]:
print(test_card)

This card is Diamond 7. The uid of this card is 20.
